In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '..')

from lib import models, graph, coarsening, utils
import tensorflow as tf
import numpy as np
import time
import scipy.sparse
from scipy.io import *
%matplotlib inline


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Graphs.
flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
# TODO: change cgcnn for combinatorial Laplacians.
flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
flags.DEFINE_integer('coarsening_levels', 4, 'Number of coarsened graphs.')

# Directories.
flags.DEFINE_string('dir_data', os.path.join('..', 'data', 'mnist'), 'Directory to store data.')

# Load Data

In [3]:
# Number of Nearest Neighbors
k = 110

base_storage_dir = '/data4/abhijeet/Datasets/PASCAL_VOC/GCN/preprocessing/experiment3/'
temp_filename = base_storage_dir + 'scikit_fc7_k_' + str(k) + '.mat'

if os.path.exists(temp_filename):
    print('File already exists\n No Preprocessing Required')
else:
    print('Do preprocessing --> TODO --> call the fucntion with a specific parameter')
    
data = scipy.io.loadmat( temp_filename)

X_train          = data['X_train']
X_test           = data['X_test']
train_labels     = data['Y_train']
test_labels      = data['Y_test']
Adjacency_test   = data['Adjacency_test']
Adjacency_train  = data['Adjacency_train']

#delete unused variables
del data

File already exists
 No Preprocessing Required


In [4]:
#define laplacians, coarsen adjacency, permutate data points.
train_laplacians = [[] for i in range(1 + FLAGS.coarsening_levels)]
test_laplacians = [[] for i in range(1 + FLAGS.coarsening_levels)]

train_perm ,test_perm = [] ,[]

for i in range(X_train.shape[0]):
    if not(i%1000):
        print(i)
    graphs, temp_perm = coarsening.coarsen(Adjacency_train[0][i], levels=FLAGS.coarsening_levels, self_connections=True)
    train_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(1 + FLAGS.coarsening_levels):
        train_laplacians[j].append(temp_L[j])     

for i in range(X_test.shape[0]):
    if not(i%1000):
        print(i)
    graphs, temp_perm = coarsening.coarsen(Adjacency_test[0][i], levels=FLAGS.coarsening_levels, self_connections=True)
    test_perm.append(temp_perm)
    temp_L = [graph.laplacian(A, normalized=True) for A in graphs]
    temp_L = [graph.rescale_L(A) for A in temp_L]
    #graph.plot_spectrum(L)
    temp_L = [A.toarray() for A in temp_L]
    for j in range(1 + FLAGS.coarsening_levels):
        test_laplacians[j].append(temp_L[j])     

#del Adjacency_test, Adjacency_train

0
1000
2000
3000
4000
5000
0
1000
2000
3000
4000


In [5]:
print(type(test_laplacians))
print(len(test_laplacians))
for i in range(4):
    print(len(test_laplacians[i]))
    print(test_laplacians[i][0].shape)

<class 'list'>
5
4952
(112, 112)
4952
(56, 56)
4952
(28, 28)
4952
(14, 14)


In [6]:
print(len(train_perm[0]),type(train_perm))
train_data = np.zeros((X_train.shape[0], len(train_perm[0]), X_train.shape[2]))
test_data = np.zeros((X_test.shape[0], len(test_perm[0]), X_test.shape[2]))
for i in range(train_data.shape[0]):
    if not(i%1000):
        print(i)
    train_data[i,:,:] = coarsening.perm_data_point(X_train[i,:,:], train_perm[i])

for i in range(test_data.shape[0]):
    if not(i%1000):
        print(i)
    test_data[i,:,:] = coarsening.perm_data_point(X_test[i,:,:], test_perm[i])

val_data = test_data.copy()
val_laplacians = test_laplacians.copy()
val_labels = test_labels.copy()

#del X_train, Y_train, X_test, Y_test 
#del perm_train, perm_test

112 <class 'list'>
0
1000
2000
3000
4000
5000
0
1000
2000
3000
4000


In [7]:
print(len(test_laplacians))
#print(test_laplacians[0])
L = []
for i in range(FLAGS.coarsening_levels):
    L.append(train_laplacians[i][0])
L = np.array(L)
print(L)
for i in range(4):
    print(L[i].shape)

5
[ array([[-0.00916375, -0.0091022 , -0.00909718, ..., -0.00908158,
        -0.00908857, -0.00909073],
       [-0.0091022 , -0.00916535, -0.00909755, ..., -0.00908056,
        -0.0090868 , -0.00907949],
       [-0.00909718, -0.00909755, -0.00916237, ..., -0.00908507,
        -0.00908112, -0.00908566],
       ..., 
       [-0.00908158, -0.00908056, -0.00908507, ..., -0.00916809,
        -0.00909857, -0.00908995],
       [-0.00908857, -0.0090868 , -0.00908112, ..., -0.00909857,
        -0.0091637 , -0.00909815],
       [-0.00909073, -0.00907949, -0.00908566, ..., -0.00908995,
        -0.00909815, -0.00916497]])
 array([[-0.01826675, -0.01819766, -0.0181865 , ..., -0.01817435,
        -0.01817092, -0.0181728 ],
       [-0.01819766, -0.01826438, -0.01819268, ..., -0.01817492,
        -0.0181736 , -0.01816924],
       [-0.0181865 , -0.01819268, -0.01825985, ..., -0.01818277,
        -0.01818589, -0.01818156],
       ..., 
       [-0.01817435, -0.01817492, -0.01818277, ..., -0.0182632 ,
   

# GCN 

In [8]:
from lib import models, graph, coarsening, utils
common = {}
common['dir_name']       = 'cifar10/'
common['num_epochs']     = 2
common['batch_size']     = 100
common['decay_steps']    = (train_data.shape[0] + val_data.shape[0]) / common['batch_size']
common['eval_frequency'] = 10 * common['num_epochs']
common['brelu']          = 'b1relu'
common['pool']           = 'mpool1'
num_labels_per_image     = 2
C = train_labels.shape[1]  # number of classes
model_perf = utils.model_perf()

In [9]:
if True:
    name = 'softmax'
    params = common.copy()
    params['dir_name'] += name
    params['regularization'] = 5e-4
    params['dropout']        = 1
    params['learning_rate']  = 0.02
    params['decay_rate']     = 0.95
    params['momentum']       = 0.9
    params['F']              = []
    params['F_0']            = 20
    params['K']              = []
    params['p']              = []
    params['M']              = [C] 
    params['train_laplacians'] = train_laplacians
    params['test_laplacians'] = val_laplacians
    params['val_laplacians'] = test_laplacians
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: logits (softmax)
    representation: M_1 = 20
    weights: M_0 * M_1 = 112 * 20 = 2240
    biases: M_1 = 20
step 20 / 100 (epoch 0.40 / 2):
  learning_rate = 2.00e-02, loss_average = 4.71e+00
4416.0 / 7013 = 0.629687722801
  validation precision: 0.43, recall : 0.42, f_measure: 0.42, mAP 0.57, MAP 0.69
  time: 33s (wall 40s)
step 40 / 100 (epoch 0.80 / 2):
  learning_rate = 2.00e-02, loss_average = 3.38e+00
5089.0 / 7013 = 0.725652359903
  validation precision: 0.54, recall : 0.61, f_measure: 0.57, mAP 0.82, MAP 0.81
  time: 34s (wall 41s)
step 60 / 100 (epoch 1.20 / 2):
  learning_rate = 2.00e-02, loss_average = 2.65e+00
5341.0 / 7013 = 0.761585626693
  validation precision: 0.52, recall : 0.67, f_measure: 0.59, mAP 0.88, MAP 0.84
  time: 35s (wall 43s)
step 80 / 100 (epoch 1.60 / 2):
  learning_rate = 2.00e-02, loss_average = 2.32e+00
5425.0 / 7013 = 0.77356338229
  validation precision: 0.54, recall : 0.69, f_measure: 0.61, mAP 0.90, MAP

In [10]:
print(np.sum(train_labels),np.sum(test_labels))

7584 7013


In [11]:
# Common hyper-parameters for networks with one convolutional layer.
common['regularization'] = 0
common['dropout']        = 1
common['learning_rate']  = 0.02
common['decay_rate']     = 0.95
common['momentum']       = 0.9
common['F']              = [10]
common['F_0']             = 20
common['K']              = [100]
common['p']              = [4]
common['M']              = [C]
common['train_laplacians'] = train_laplacians
common['test_laplacians'] = val_laplacians
common['val_laplacians'] = test_laplacians

In [12]:
# With 'chebyshev2' and 'b2relu', it corresponds to cgcnn2_2(L[0], F=10, K=20).
if True:
    name = 'cgconv_softmax'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
#    params['filter'] = 'chebyshev2'
#    params['brelu'] = 'b2relu'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 112 * 10 / 4 = 280
    weights: F_0 * F_1 * K_1 = 20 * 10 * 100 = 20000
    biases: F_1 = 10
  layer 2: logits (softmax)
    representation: M_2 = 20
    weights: M_1 * M_2 = 280 * 20 = 5600
    biases: M_2 = 20
0 th conv layer 100 112 112
step 20 / 100 (epoch 0.40 / 2):
  learning_rate = 2.00e-02, loss_average = 4.16e+00
3932.0 / 7013 = 0.560673035791
  validation precision: 0.28, recall : 0.34, f_measure: 0.31, mAP 0.49, MAP 0.63
  time: 22s (wall 25s)
step 40 / 100 (epoch 0.80 / 2):
  learning_rate = 2.00e-02, loss_average = 3.47e+00
4512.0 / 7013 = 0.64337658634
  validation precision: 0.37, recall : 0.47, f_measure: 0.41, mAP 0.61, MAP 0.73
  time: 28s (wall 31s)
step 60 / 100 (epoch 1.20 / 2):
  learning_rate = 2.00e-02, loss_average = 2.85e+00
4882.0 / 7013 = 0.696135747897
  validation precision: 0.51, recall : 0.54, f_measure: 0.53, mAP 0.71, MAP 0.78
  time: 33s (wall 36s)
step 80 / 10

In [13]:
# Common hyper-parameters for LeNet5-like networks.
common['regularization'] = 5e-4
common['dropout']        = 0.5
common['learning_rate']  = 0.02  # 0.03 in the paper but sgconv_sgconv_fc_softmax has difficulty to converge
common['decay_rate']     = 0.95
common['momentum']       = 0.9
common['F']              = [32, 64]
common['F_0']            = 20
common['K']              = [25, 25]
common['p']              = [2, 2]
common['M']              = [512, C]
common['train_laplacians'] = train_laplacians
common['test_laplacians'] = val_laplacians
common['val_laplacians'] = test_laplacians

In [14]:
if True:
    name = 'cgconv_cgconv_fc_softmax'  # 'Chebyshev'
    params = common.copy()
    params['dir_name'] += name
    params['filter'] = 'chebyshev5'
    model_perf.test(models.cgcnn(L, num_labels_per_image, **params), name, params,
                    train_data, train_labels, val_data, val_labels, test_data, test_labels)

NN architecture
  input: M_0 = 112
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 112 * 32 / 2 = 1792
    weights: F_0 * F_1 * K_1 = 20 * 32 * 25 = 16000
    biases: F_1 = 32
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 56 * 64 / 2 = 1792
    weights: F_1 * F_2 * K_2 = 32 * 64 * 25 = 51200
    biases: F_2 = 64
  layer 3: fc1
    representation: M_3 = 512
    weights: M_2 * M_3 = 1792 * 512 = 917504
    biases: M_3 = 512
  layer 4: logits (softmax)
    representation: M_4 = 20
    weights: M_3 * M_4 = 512 * 20 = 10240
    biases: M_4 = 20
0 th conv layer 100 112 112
1 th conv layer 100 56 56
step 20 / 100 (epoch 0.40 / 2):
  learning_rate = 2.00e-02, loss_average = 8.68e+00
2426.0 / 7013 = 0.34592898902
  validation precision: 0.07, recall : 0.10, f_measure: 0.09, mAP 0.10, MAP 0.40
  time: 5s (wall 5s)
step 40 / 100 (epoch 0.80 / 2):
  learning_rate = 2.00e-02, loss_average = 7.23e+00
2424.0 / 7013 = 0.345643804363
  validation precision: 0.02, recall : 0.10, f_me